<img src="https://teaching.bowyer.io/SDSAI/0/img/IMPERIAL_logo_RGB_Blue_2024.svg" alt="Imperial Logo" width="500"/><br /><br />

Supervised Learning
==============
### SURG70098 - Surgical Data Science and AI
### Stuart Bowyer

## Intended Learning Outcomes
1.  U


## Session Outline
1.  [Introduction to Supervised Machine Learning](#introduction_to_supervised_machine_learning)
1.  [Regression Models](#regression_models)
1.  [Classification Models](#classification_models)
1.  [Model Reliability and Reproducibility](#model_reliability_and_reproducibility)
1.  [Wrap Up](#wrap_up)

# Introduction to Supervised Machine Learning

## What is Supervised Learning

Creating a model based on a labelled set of data that can be used to predict something about future (unlabelled) data

## Stages in the Supervised Learning Process
1.  (Data Acquisition)
1.  (Data Preparation)
1.  Training
1.  Validation/Evaluation
1.  Deployment/Use

## Regression vs Classification

## Example Uses of Supervised Learning

| Regression | Classification |
| ---------- | -------------- |
| aaaa | Predict risk of postoperative pulmonary complications (Logistic Regression/Boosted Trees) - [link](https://doi.org/10.1016/S2589-7500(24)00065-7) |
| Predict FEV1 and FVC from chest x-ray (Deep NN) - [link](https://doi.org/10.1016/S2589-7500(24)00113-4) | Identify (and rule out) patients with suspected myocardial infarction (Logistic Regression) - [link](https://doi.org/10.1016/S2589-7500(24)00191-2) |
| aaaa | aaaa |
| aaaa | aaaa |



# Regression Models

## Linear Regression

## Non-linear Regression

## Model Evaluation
How do you know if your regression model is a good predictor?

### Metrics
incl. Stats analysis for model goodness of fit (residual plot, QQ plot)

### (Revisiting) Normalisation etc.
Demonstrate explain why these are helpful

## Validation

### Test / Train Splitting

### Cross Validation

## Regularisation

## Exercise

# Classification Models

## Logistic Regression

## SVM

## Decision Trees

## Model Evaluation

## Exercise

# Model Reliability and Reproducibility

## Overfitting and Underfitting

## Bias and Variance

## Hyperparameter Tuning

## Learning Curves

## Sensitivity Analysis

# Tutorial Exercise

# Wrap Up
*   ???

## Before Next Session

*   ???

### New Material
*   Regularisation methods
*   Boosted trees (ensemble methods)

### Consolidation Reading
*   ???